In [1]:
!pip install beautifulsoup
!pip install urllib
!pip install pandas

In [2]:
# 필요한 라이브러리 import
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [11]:
# 크롤링 결과를 저장할 리스트
date_list = []
title_list = []
press_list = []
contents_list = []
link_list = []
data_dict = {}

In [ ]:
# 기사의 내용을 정제화하는 함수 
def contents_cleansing(contents):
    # 앞에 있는 내용 중 필요없는 부분을 제거한다
    first_step = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', str(contents)).strip()
    # 뒤에 있는 내용 중 필요없는 부분을 제거한다
    second_step = re.sub('<ul class="relation_lst">.*?</dd>', '', first_step).strip()
    third_step = re.sub('<.+?>', '', second_step).strip()
    # 정제화를 마친 내용을 기사 내용 담는 list에 append해준다.
    contents_list.append(third_step)
    

def crawling(maxpage, keyword, sort, start, end):
    page = 1
    max =(int(maxpage) - 1) * 10 + 1
    d_from = start.replace(".","")
    d_to = end.replace(".","")
    
    while page <= max:
        url = "https://search.naver.com/search.naver?where=news&query=" + keyword + "&sort=" + sort + "&ds=" + start + "&de=" + end + "&nso=so%3Ar%2Cp%3Afrom" + d_from + "to" + d_to + "%2Ca%3A&start=" + str(page)
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
 
        # <a> 태그에서 제목과 링크를 추출한다.
        a_tag = soup.select('.news_tit')
        for i in a_tag:
            # 기사 제목 추출 
            title_list.append(i.text)
            # 기사 링크 추출
            link_list.append(i['href'])
            
        # 신문사를 추출한다.
        press_lists = soup.select('.info_group > .press')
        for i in press_lists:
            press_list.append(i.text)
        
        # 기사 날짜를 추출한다.
        date_lists = soup.select('.info_group > span.info')
        for i in date_lists:
            # n면 n단에서 '면'을 제거하여 숫자만을 추출한다.
            if i.text.find("면") == -1:
                date_list.append(i.text)
        
        # 기사 요약본을 추출한다.
        contents_lists = soup.select('.news_dsc')
        for i in contents_lists:
            # 기사 요약본을 정제화할 함수를 호출한다.
            contents_cleansing(i)
        
        # 리스트 형태로 저장된 data, title, press, contents, link를 data_dict에 딕셔너리 형태로 넣어준다.
        data_dict= {"date" : date_list , "title":title_list ,  "press" : press_list ,"contents": contents_list ,"link":link_list }  
        print(page)
        
        # 딕셔너리로 저장된 data_dict를  데이터프레임으로 변환해준다.
        df = pd.DataFrame(data_dict)
        page += 10
        df
    
    
    # to_csv를 사용하여 dataframe을 csv으로 생성
    outputFileName = 'doosanbears_11'
    df.to_csv("doosanbears_11.csv")
    

def main():
    maxpage = input("최대 페이지 수: ")  
    keyword = input("키워드: ")  
    sort = input("관련도순=0  최신순=1  오래된순=2: ")
    start = input("시작 날짜 ex.(2021.04.03):")
    end = input("마지막 날짜 ex.(2021.11.18):")
    
    crawling(maxpage, keyword, sort, start, end) 
    
main()